<a href="https://colab.research.google.com/github/praveenmec67/FakeCurrencyDetection/blob/master/Classifier1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as im
import boto3
import os
import cv2 as cv2
import tempfile
from tensorflow.keras import layers,models

In [3]:
img_width=224
img_height=224
channels=3
l=[]
empty=np.empty([1,224,224,3])

ACCESS_KEY = 'XXXXXXXXXXXXXXX'
SECRET_KEY = 'XXXXXXXXXXXXXXXXXXXXXX'


In [4]:
def visualize(img,blur,medianfil,rotate,brightness,contrast):

   fig = plt.figure(figsize=(20,20))
   fig.subplots_adjust(wspace=0.5)

  
   plt.subplot(1,7,1)
   plt.title('Original')
   plt.imshow(img)

   plt.subplot(1,7,2)
   plt.title('Blur')
   plt.imshow(blur)


   plt.subplot(1,7,3)
   plt.title('MedianFilter')
   plt.imshow(medianfil)

   plt.subplot(1,7,4)
   plt.title('Rotate')
   plt.imshow(rotate)

 #  plt.subplot(1,7,5)
 #  plt.title('GrayScale')
 #  plt.imshow(gray)

   plt.subplot(1,7,5)
   plt.title('Brightness')
   plt.imshow(brightness)

   plt.subplot(1,7,6)
   plt.title('Contrast')
   plt.imshow(contrast)

In [5]:
def image_aug(img):
  
  
  rows,cols = img.shape[0],img.shape[1]
  b,c = np.random.uniform(0.2,0.6),np.random.uniform(0.2,0.6)

  img_org=img
  img_org=img.reshape(1,img_width,img_height,channels)
  img_org=tf.convert_to_tensor(img_org,dtype=tf.uint8)



  gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
  #gray=gray.reshape(1,img_width,img_height,channels)
  #gray=tf.convert_to_tensor(gray,dtype=tf.uint8)
  
  
  blur=cv2.blur(img,(5,5))
  blur=blur.reshape(1,img_width,img_height,channels)
  blur=tf.convert_to_tensor(blur,dtype=tf.uint8)


  medianfil=cv2.medianBlur(img,5)
  medianfil=medianfil.reshape(1,img_width,img_height,channels)
  medianfil=tf.convert_to_tensor(medianfil,dtype=tf.uint8)


  brightness=tf.image.adjust_brightness(img,b)
  brightness=tf.reshape(img,[1,img_width,img_height,channels])


  contrast=tf.image.adjust_contrast(img,c)
  contrast=tf.reshape(img,[1,img_width,img_height,channels])



  for i in np.random.randint(0,360,1):
    M = cv2.getRotationMatrix2D((cols/2,rows/2),i,1)
    rotate = cv2.warpAffine(img,M,(cols,rows))
    rotate=rotate.reshape(1,rotate.shape[0],rotate.shape[1],rotate.shape[2])
    rotate=tf.convert_to_tensor(rotate,dtype=tf.uint8)


    img_aug=np.concatenate([img_org,blur,medianfil,brightness,contrast,rotate],axis=0)
    return img_aug
 

  #visualize(img,blur,medianfil,rotate,brightness,contrast)

In [ ]:

s3=boto3.resource('s3',aws_access_key_id=ACCESS_KEY,aws_secret_access_key=SECRET_KEY)
bucket=s3.Bucket('fakedetection')
ignore='Denomination_Original/'


for i in bucket.objects.filter(Prefix='Denomination_Original/'):
  if i.key !=ignore:
   l.append(i.key)
  else:
    continue
print('Total Images in S3 bucket: '+str(len(l)))


total=0
for j in l:
  if total==len(l):
    break
  else:
    name='img_'+str(total)
    key=j
    obj=bucket.Object(key)
    tmp = tempfile.NamedTemporaryFile()

    with open(tmp.name, 'wb') as f:
      obj.download_fileobj(f)
      img=im.imread(tmp.name)
      img=cv2.resize(img,(img_width,img_height),interpolation=cv2.INTER_LINEAR)
      
      print(j +' : '+str(img.shape))
      images_aug=image_aug(img)
      final_inp=np.append(empty,images_aug,0)
      empty=final_inp
      total=total+1
      

final_inp=np.delete(final_inp,[0],0)


print('Total Images is : '+str(total))
print('Total Images Shape : '+str(final_inp.shape))

In [ ]:
s3=boto3.resource('s3',aws_access_key_id=ACCESS_KEY,aws_secret_access_key=SECRET_KEY)
bucket=s3.Bucket('fakedetection')
ignore='Labels/'
list=[]
for i in bucket.objects.filter(Prefix='Labels/'):
  if i.key!=ignore:
    list.append(i.key)
    print(list)
  else:
    continue
for j in list:
    key=j
    print(key)
    obj=bucket.Object(key)
    print(obj)
    tmp = tempfile.NamedTemporaryFile()

    with open(tmp.name, 'wb') as f:
      obj.download_fileobj(f)
      labels_inp=pd.read_csv(tmp.name).iloc[:,1].values
      print(labels_inp)

  

In [8]:
#X=tf.reshape(final_inp,shape=[1002,(224*224*3)])
final_inp=final_inp/255.
train_images=final_inp[:900,:]
test_images=final_inp[900:,:]
train_labels=labels_inp[:900]
test_labels=labels_inp[900:]




In [ ]:

model=tf.keras.models.Sequential([
  tf.keras.layers.Convolution2D(32,(3,3),(2,2),input_shape=(img_width,img_height,3)),
  tf.keras.layers.Convolution2D(64,(5,5),(1,1)),
  tf.keras.layers.Activation(activation='relu'),
  tf.keras.layers.MaxPooling2D((2,2),(1,1)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(1,activation='sigmoid')
])
model.summary()
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics='accuracy')
history=model.fit(train_images,train_labels,epochs=5,validation_split=0.2,shuffle='y')

In [ ]:
model1=tf.keras.models.Sequential([
  tf.keras.layers.Convolution2D(32,(3,3),(2,2),input_shape=(img_width,img_height,3)),
  tf.keras.layers.Convolution2D(64,(5,5),(1,1)),
  tf.keras.layers.Activation(activation='relu'),
  tf.keras.layers.MaxPooling2D((2,2),(1,1)),
  tf.keras.layers.Convolution2D(512,(3,3),(2,2)),
  tf.keras.layers.Activation(activation='relu'),
  tf.keras.layers.Convolution2D(64,(5,5),(1,1)),
  tf.keras.layers.MaxPooling2D((2,2),(1,1)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(1,activation='sigmoid')
])
model1.summary()
model1.compile(optimizer='Adam',loss='binary_crossentropy',metrics='accuracy')
history=model1.fit(train_images,train_labels,epochs=5,validation_split=0.2,shuffle='y')

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
test_loss,test_acc=model.evaluate(np.asarray(test_images),np.asarray(test_labels),verbose=0)
print(test_loss)
print(test_acc)


In [ ]:
empty1=np.empty([1,224,224,3])
s3=boto3.resource('s3',aws_access_key_id=ACCESS_KEY,aws_secret_access_key=SECRET_KEY)
bucket=s3.Bucket('fakedetection')
ignore='Test/'
list=[]
for i in bucket.objects.filter(Prefix='Test/'):
  if i.key!=ignore:
    list.append(i.key)
  else:
    continue

print('Total Images in S3 bucket: '+str(len(list)))


total=0
for j in list:
  if total==len(list):
    break
  else:
    key=j
    print(key)
    obj=bucket.Object(key)
    tmp = tempfile.NamedTemporaryFile()

    with open(tmp.name, 'wb') as f:
      obj.download_fileobj(f)
      img=im.imread(tmp.name)
      img=cv2.resize(img,(img_width,img_height),interpolation=cv2.INTER_LINEAR)
      img=img/255
      img=img.reshape(1,img_width,img_height,channels)
      test_inp=np.append(empty1,img,0)
      empty1=test_inp
      total=total+1
      



print(test_inp.shape)
print('Total Images is : '+str(total))
print('Total Images Shape : '+str(test_inp.shape))
      

  

In [ ]:
print(test_inp)
test=np.delete(test_inp,[0],0)
print(test)

In [ ]:
pred=model1.predict(test).flatten().tolist()
print(pred)
